In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, get_json_object, collect_list

In [2]:
spark = SparkSession.builder \
    .appName("ol_author") \
    .getOrCreate()

23/08/01 19:52:21 WARN Utils: Your hostname, Stephanies-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.4.152 instead (on interface en0)
23/08/01 19:52:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/01 19:52:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/01 19:52:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
file_path = "./data/ol_dump_authors_2023-06-30.txt"
ol_author_df = spark.read.text(file_path)
ol_author_df.show()

+--------------------+
|               value|
+--------------------+
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
|/type/author\t/au...|
+--------------------+
only showing top 20 rows



In [4]:
split_col = split(ol_author_df["value"], "\t")

In [5]:
ol_author_df = ol_author_df.withColumn("type", split_col.getItem(0))
ol_author_df = ol_author_df.withColumn("author_key", split_col.getItem(1))
ol_author_df = ol_author_df.withColumn("revision", split_col.getItem(2))
ol_author_df = ol_author_df.withColumn("timestamp", split_col.getItem(3))
ol_author_df = ol_author_df.withColumn("json_data", split_col.getItem(4))



In [6]:
ol_author_df.show()

+--------------------+------------+--------------------+--------+--------------------+--------------------+
|               value|        type|          author_key|revision|           timestamp|           json_data|
+--------------------+------------+--------------------+--------+--------------------+--------------------+
|/type/author\t/au...|/type/author|/authors/OL10000175A|       1|2021-12-26T21:34:...|{"type": {"key": ...|
|/type/author\t/au...|/type/author|/authors/OL10000203A|       1|2021-12-26T21:36:...|{"type": {"key": ...|
|/type/author\t/au...|/type/author|/authors/OL10000507A|       1|2021-12-26T22:29:...|{"type": {"key": ...|
|/type/author\t/au...|/type/author|/authors/OL10000910A|       1|2021-12-26T23:32:...|{"type": {"key": ...|
|/type/author\t/au...|/type/author| /authors/OL1000146A|       2|2008-08-20T17:57:...|{"name": "Ka\u030...|
|/type/author\t/au...|/type/author|/authors/OL10001571A|       1|2021-12-27T01:18:...|{"type": {"key": ...|
|/type/author\t/au...|/type/

In [7]:
ol_author_df.columns

['value', 'type', 'author_key', 'revision', 'timestamp', 'json_data']

In [8]:
json_author_data = ol_author_df.select("json_data")

In [20]:
# Extract the 'name' field from the JSON data using get_json_object
ol_author_df = ol_author_df.withColumn("author_name", get_json_object(ol_author_df["json_data"], "$.name"))
ol_author_df.columns

['value',
 'type',
 'author_key',
 'revision',
 'timestamp',
 'json_data',
 'author_name']

In [10]:
author_key_df = ol_author_df.select("author_key","author_name")
author_key_df.show()

+--------------------+--------------------+
|          author_key|         author_name|
+--------------------+--------------------+
|/authors/OL10000175A| angel-rose sandoval|
|/authors/OL10000203A|     Thomas Stippich|
|/authors/OL10000507A|Maximilian Stefan...|
|/authors/OL10000910A|Beckel, James A.,...|
| /authors/OL1000146A|    Kāẓim Ḥajjāj|
|/authors/OL10001571A|     Don J. Champion|
|/authors/OL10001765A|        Norfrid Klug|
| /authors/OL1000183A|Ḥammūd Ḥammād...|
|/authors/OL10002260A|       Andrea Berndt|
|/authors/OL10002453A|      Sarajane Trier|
|/authors/OL10002467A|   Laura Laura Wayth|
|/authors/OL10002472A|        Yuri M. Rios|
|/authors/OL10002982A|           Karl Haag|
|/authors/OL10003680A|          Rhodes Tom|
|/authors/OL10003698A|        Jurgen Ebert|
|/authors/OL10003792A|    Martina Ankarfyr|
|/authors/OL10003851A|           Elle King|
|/authors/OL10004199A|          Pat Murphy|
|/authors/OL10004342A|  Merve Urem Cetinel|
|/authors/OL10004348A|          

In [21]:
author_df = ol_author_df.drop("value", "type", "revision", "timestamp","json_data")
author_df.show()

+--------------------+--------------------+
|          author_key|         author_name|
+--------------------+--------------------+
| /authors/OL9072096A|   Bruce Alan Ekholt|
| /authors/OL2313307A|Universidad de Se...|
| /authors/OL7442871A|Salt Lake Oratori...|
| /authors/OL5538948A|Muḥammad ibn Musl...|
| /authors/OL1193645A|Dahlem Workshop o...|
| /authors/OL7073368A| Fernando Pérez Royo|
| /authors/OL6060113A|Deutsche National...|
| /authors/OL9609646A|     R. D. . Gardner|
| /authors/OL9389082A|    Jeremiah Spencer|
| /authors/OL4636498A|    D. A. Kiristutās|
| /authors/OL2725013A|           Ed Flower|
| /authors/OL4644223A|       Armin Germann|
| /authors/OL2096583A|Doulton and Company.|
|/authors/OL10395947A|         Bill Siever|
| /authors/OL2516399A|   Dabney, Virginius|
| /authors/OL6363387A| Matthias Elsdörfer|
|/authors/OL10455335A|     Carla Raffaelli|
| /authors/OL1576521A|      Pippo Carrubba|
| /authors/OL3760852A|Tommie L. Watkins...|
| /authors/OL7006118A|    Daniel

In [24]:
output_path = "./data/author_dataframe"

author_df = author_df.repartition(1)

# Write the DataFrame to Parquet format
author_df.write.parquet(output_path)

In [25]:
# output_path = "./data/ol_author_key.csv"
# # Write the Spark DataFrame to a CSV file
# author_key_df.write.option("header", "true").csv(output_path)